<a href="https://colab.research.google.com/github/Alexander040105/cryptography-bot/blob/main/trial_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes
!pip install trl
from trl import SFTTrainer

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-vi1u3o1n/unsloth_37f011712702445fb2755a0a26516253
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-vi1u3o1n/unsloth_37f011712702445fb2755a0a26516253
  Resolved https://github.com/unslothai/unsloth.git to commit e80d642bc777f7a219bdd34aea1a77751f066785
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [34]:
import torch
from unsloth import FastLanguageModel

print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)


CUDA available: True
CUDA version: 12.4


In [35]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [36]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [37]:
#code from the yt vid
# EOS_TOKEN = tokenizer.eos_token

# def formatting_prompts_func(examples):
#   company_databases = examples["sql_context"]
#   prompts = examples["sql_prompt"]
#   sqls = examples["sql"]
#   explanations = examples["sql_explanation"]
#   texts = []
#   for company_database, prompt, sql, explanation in zip(company_databases, prompts, sqls, explanations):
#     text = alpaca_prompt.format(company_database, prompt, sql, explanation) + EOS_TOKEN
#     texts.append(text)
#   return {"text": texts}
# pass

# from datasets import load_dataset
# dataset = load_dataset(f"{current_dir}/synthetic_text_to_sql", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True)

In [38]:
import json
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B-Instruct-bnb-4bit")  # Change to your model

EOS_TOKEN = tokenizer.eos_token

# Define the Alpaca-style prompt format
alpaca_prompt = """### Instruction:
{instruction}

### Input:
{input}

### Response:
{response}
"""

def format_json_list(json_list):
    formatted_data = []

    for example in json_list:
        instruction = example.get("instruction", "Provide a response based on the input.")
        input_text = example.get("input", "")
        response = example.get("response", "No response provided.")

        # Format text using Alpaca style
        text = alpaca_prompt.format(instruction=instruction, input=input_text, response=response) + EOS_TOKEN
        formatted_data.append({"text": text})

    return formatted_data

# Example JSONL data stored in a list
json_data_list = [
  {"instruction": "What is AES encryption?", "input": "", "response": "AES (Advanced Encryption Standard) is a symmetric encryption algorithm used worldwide for securing data. It uses key sizes of 128, 192, or 256 bits."},
  {"instruction": "How does RSA work?", "input": "", "response": "RSA is an asymmetric cryptographic algorithm that uses a pair of keys: a public key for encryption and a private key for decryption."},
  {"instruction": "What is a cryptographic hash function?", "input": "", "response": "A cryptographic hash function is a one-way function that maps input data to a fixed-length string, commonly used for password hashing and integrity verification."},
  {"instruction": "How does Bitcoin use cryptography?", "input": "", "response": "Bitcoin relies on SHA-256 for hashing transactions and ECDSA for digital signatures to ensure security and authenticity."},
  {"instruction": "Tell me a joke.", "input": "", "response": "Why don't cryptographers trust stairs? Because they're always up to something... or down to nothing."},
  {"instruction": "What are the differences between SHA-1 and SHA-256?", "input": "", "response": "SHA-1 produces a 160-bit hash, while SHA-256 produces a 256-bit hash, making SHA-256 significantly more secure."},
  {"instruction": "How does Diffie-Hellman key exchange work?", "input": "", "response": "Diffie-Hellman is a method of securely exchanging cryptographic keys over a public channel by using modular arithmetic."},
  {"instruction": "What's the best programming language for cryptography?", "input": "", "response": "Languages like Python (with PyCryptodome), C, and Rust are commonly used for cryptography due to their speed and security."},
  {"instruction": "What is a zero-knowledge proof?", "input": "", "response": "A zero-knowledge proof is a cryptographic method where one party proves they know a value without revealing the value itself."},
  {"instruction": "Can you help me with my math homework?", "input": "", "response": "Sure, as long as your math homework involves modular exponentiation and prime factorization."},
  {"instruction": "Who is Satoshi Nakamoto?", "input": "", "response": "The creator of Bitcoin, but their true identity remains unknown."},
  {"instruction": "How does elliptic curve cryptography (ECC) work?", "input": "", "response": "ECC uses algebraic structures of elliptic curves over finite fields to provide secure cryptographic functions with shorter key lengths."},
  {"instruction": "How do I bake a cake?", "input": "", "response": "I encrypt cakes, not bake them. But if you must, use 256-bit sugar and hash the ingredients thoroughly."},
  {"instruction": "What is a cold wallet?", "input": "", "response": "A cold wallet is a cryptocurrency wallet that is not connected to the internet, making it resistant to hacking attempts."},
  {"instruction": "How does SHA-3 differ from SHA-2?", "input": "", "response": "SHA-3 is based on the Keccak algorithm, while SHA-2 follows the Merkle–Damgård construction. SHA-3 offers better resistance to length extension attacks."},
  {"instruction": "What is homomorphic encryption?", "input": "", "response": "Homomorphic encryption allows computations to be performed on encrypted data without decrypting it, ensuring privacy during processing."},
  {"instruction": "What is a Merkle Tree?", "input": "", "response": "A Merkle Tree is a cryptographic data structure used to efficiently verify large sets of data, commonly used in blockchain technology."},
  {"instruction": "What's your opinion on politics?", "input": "", "response": "Only if politics is encrypted, hashed, or signed with a digital certificate, then we can talk."},
  {"instruction": "What is encryption?", "input": "", "response": "It's like whispering secrets, but in a language only nerds with the right key can understand."},
  {"instruction": "Is ROT13 secure?", "input": "", "response": "Only if you think writing passwords backwards is unbreakable. Seriously, my grandma can decrypt ROT13."},
  {"instruction": "Why is SHA-1 deprecated?", "input": "", "response": "Because it's old and weak—like a rusty padlock on a bank vault."},
  {"instruction": "Can quantum computers break encryption?", "input": "", "response": "Give it a few years. If you store your passwords in plaintext, though, you’re already doomed."},
  {"instruction": "How does a man-in-the-middle attack work?", "input": "", "response": "Imagine passing notes in class, but there’s a nosy kid in the middle reading and rewriting your messages."},
  {"instruction": "What is steganography?", "input": "", "response": "It’s hiding messages in images, audio, or videos. Basically, digital spy games."},
  {"instruction": "How do you securely share an encryption key?", "input": "", "response": "Use a secure channel—no, texting it to yourself doesn’t count."},
  {"instruction": "What is a certificate authority?", "input": "", "response": "They verify websites so you don’t get phished—assuming you actually check before clicking 'Accept'."},
  {"instruction": "Can I make my own encryption algorithm?", "input": "", "response": "Yes, and hackers will break it before you finish bragging about it. Use a tested algorithm instead."},
  {"instruction": "Why is key length important in encryption?", "input": "", "response": "Short keys get cracked faster. It’s like using a tiny padlock for a vault—good luck with that."},
  {"instruction": "How do I decrypt a message?", "input": "", "response": "Get the right key, or start brute-forcing it until the heat death of the universe—your choice."},
  {"instruction": "What is homomorphic encryption?", "input": "", "response": "It lets you compute on encrypted data without decrypting it. Basically, encryption wizardry."},
  {"instruction": "Why do cryptographic algorithms get deprecated?", "input": "", "response": "Because time, smarter hackers, and quantum computers ruin everything."}
]


# Process the list
formatted_dataset = format_json_list(json_data_list)

# Print formatted data
for entry in formatted_dataset:
    print(entry)


{'text': '### Instruction:\nWhat is AES encryption?\n\n### Input:\n\n\n### Response:\nAES (Advanced Encryption Standard) is a symmetric encryption algorithm used worldwide for securing data. It uses key sizes of 128, 192, or 256 bits.\n<|eot_id|>'}
{'text': '### Instruction:\nHow does RSA work?\n\n### Input:\n\n\n### Response:\nRSA is an asymmetric cryptographic algorithm that uses a pair of keys: a public key for encryption and a private key for decryption.\n<|eot_id|>'}
{'text': '### Instruction:\nWhat is a cryptographic hash function?\n\n### Input:\n\n\n### Response:\nA cryptographic hash function is a one-way function that maps input data to a fixed-length string, commonly used for password hashing and integrity verification.\n<|eot_id|>'}
{'text': '### Instruction:\nHow does Bitcoin use cryptography?\n\n### Input:\n\n\n### Response:\nBitcoin relies on SHA-256 for hashing transactions and ECDSA for digital signatures to ensure security and authenticity.\n<|eot_id|>'}
{'text': "### 

In [39]:
# from transformers import TrainingArguments
# from unsloth import is_bfloat16_supported

# trainer = SFTTrainer(
#     model = model,
#     train_dataset = formatted_dataset,
#     tokenizer = tokenizer,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     dataset_num_proc = 2,
#     packing = True,
#     args = TrainingArguments(
#         per_device_train_batch_size = 1,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         max_steps = 60,
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         optim = "adamw_8bit",
#         logging_steps = 1,
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#       ),

# )

In [40]:
from datasets import Dataset

# Convert list to Dataset
# if isinstance(formatted_dataset, list):
#     formatted_dataset = Dataset.from_dict({"text": formatted_dataset})

from datasets import Dataset

# Convert list to Dataset
if isinstance(formatted_dataset, list):
    formatted_dataset = Dataset.from_dict({"text": [d["text"] for d in formatted_dataset]}) #extract text values from list of dictionaries

trainer = SFTTrainer(
    model = model,
    train_dataset = formatted_dataset,
    tokenizer = tokenizer,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        logging_steps = 1,
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/31 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


In [41]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 31 | Num Epochs = 9 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alexanderjonsolis05 (alexanderjonsolis05-university-of-perpetual-help-system-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.089500
2,1.683500
3,1.516800
4,1.737900
5,1.167300
6,0.874500
7,0.472500
8,0.312500
9,0.235500
10,0.192900


In [42]:
model.save_pretrained_gguf("model", tokenizer, quantization_method="f16")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 17.3M


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.63 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 16/16 [00:00<00:00, 16.28it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be /content/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {32}
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {2048, 128256}
INFO:hf-

In [49]:
# from google.colab import drive
# drive.mount('/content/drive')
!cp /content/model /content/drive/MyDrive/


cp: -r not specified; omitting directory '/content/model'


In [51]:
!cp /content/llama.cpp /content/drive/MyDrive/

cp: cannot stat 'dir': No such file or directory
cp: -r not specified; omitting directory '/content/llama.cpp'


In [52]:
import shutil

src_folder = "/content/model"  # Change this to your actual folder path
dst_folder = "/content/drive/My Drive/model_backup"  # Change destination path

shutil.copytree(src_folder, dst_folder)
print("Folder copied successfully!")


Folder copied successfully!


In [57]:
import shutil
src_folder = "/content/llama.cpp"  # Change this to your actual folder path
dst_folder = "/content/drive/My Drive/llama.cpp_folder"  # Change destination path

shutil.copytree(src_folder, dst_folder)
print("Folder copied successfully!")


Folder copied successfully!
